# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

# Генерация

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter
import nltk
from nltk.tokenize import sent_tokenize
from scipy.sparse import lil_matrix
from numpy import array, matrix

In [2]:
dvach = open('2ch_corpus.txt', encoding="utf8").read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [4]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] + ['<end>'] for text in sent_tokenize(dvach[:5000000])]

In [5]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0, len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [6]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence, 2))
    trigrams_dvach.update(ngrammer(sentence, 3))

In [7]:
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                           len(unigrams_dvach)))


id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}


for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = ' '.join([word1, word2])
    matrix_dvach[bigram2id_dvach[bigram], word2id_dvach[word3]] = (trigrams_dvach[ngram]/bigrams_dvach[bigram])


In [10]:
def generate(matrix, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    current_bigram = start
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen])
        current_bigram = f'{current_bigram.split()[1]} {id2word[chosen]}'
        
        if id2word[chosen] == '<end>':
            chosen = bigram2id[start]
            current_bigram = start
            
        current_idx = bigram2id[current_bigram]
    
    return ' '.join(text)

In [25]:
print(generate(matrix_dvach, id2word_dvach, bigram2id_dvach).replace('<end>', '\n'))

оп насчёт пользы просмотра фильмов тарковского городские реднеки верят им и бегут обратно к тянке встретил ее в посадке потом и кол оф дунью затравим 
 цыгане возьмут ракку 
 лол баста недавно попал в компанию 
 я вот неделю назад обновился до 1 урезали 
 кризис произойдет в любом случае всё не просто так мамку анона ебал и так переполнен всяким говном а какме должны быть типа путешествие во времени в 2007 метолкорщики давно гриву сбрили 
 да ему похуй ирл так и познакомилися 
 n по 50000 долларов платят а тесла-кар стоит 200 баксов 12 
 поэтому я


# Перплексия

In [31]:
probas_dvach = [normalize(text) for text in sent_tokenize(dvach[-3000:])][0:39]
text_generated = generate(matrix_dvach, id2word_dvach, bigram2id_dvach).replace('<end>', '\n').split('\n')[0:3]

In [36]:
def compute_joint_proba_unigrams(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)


def compute_join_proba_markov_assumption_bigrams(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>'], 2):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)


def compute_join_proba_markov_assumption_trigrams(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>'], 3):
        word1, word2, word3 = ngram.split()
        bigram = ' '.join([word1, word2])
        if bigram in bigram_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

In [37]:
perplexity_unigrams = [perplexity(*compute_joint_proba_unigrams(i, probas_dvach)) for i in text_generated]
perplexity_bigrams = [perplexity(*compute_join_proba_markov_assumption_bigrams(i, unigrams_dvach, bigrams_dvach)) for i in text_generated]
perplexity_trigrams = [perplexity(*compute_join_proba_markov_assumption_trigrams(i, bigrams_dvach, trigrams_dvach)) for i in text_generated]

In [38]:
print(f'Перплексия униграмной модели - {np.mean(perplexity_unigrams)}')
print(f'Перплексия биграмной модели - {np.mean(perplexity_bigrams)}')
print(f'Перплексия триграмной модели - {np.mean(perplexity_trigrams)}')

Перплексия униграмной модели - 5000.000000000001
Перплексия биграмной модели - 228.3861221437613
Перплексия триграмной модели - 3.471894402836019


Вывод: чем больше мы учитываем контекст, тем меньше перплексия (и тем более осмысленным получается текст)

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Если нам необходимо учесть, что слова могут не встречаться в предзаданном словаре, то мы создаём т.н. "систему открытого словаря", добавляя тег <UNK>(неизвестный). В некоторых системах любое слово, которого нет в словаре, превращается в <UNK>, и затем его вероятность считается как для любого другого слова в тренировочном сете данных. 

Другой вариант - мы не берём словарь заранее, но создаём его по мере работы. В таком случае слова заменяются на <UNK> в зависимости от частоты. Но необходимо определить размер словаря заранее, чтобы определить, насколько редким должно быть слово, чтобы мы могли заменить его на <UNK>.

Выбор модели <UNK> влияет на перплексию и подобные ей метрики. Низкая перплексия может получиться, например, если взять маленький словарь и приписать <UNK> большую вероятность. Поэтому перплексию стоит использовать для моделей с одинаковым словарём.

2. Что такое сглаживание (smoothing)?

Сглаживание необходимо, чтобы учесть слова, которые не встречаются в тренировочной выборке (но которые могут появиться в тестовой). Как правило, мы отнимаем немного вероятности у самых частотных слов и добавляем неизвестным.

Сглаживание Лапласа (сглаживание +1) - добавить единицу ко всем н-граммам перед расчётом вероятности. Не используется в современных н-грамных моделях, но для классификации текста всё ещё подходит.

Сглаживание +к - добавить к событиям, которые пока не встречались,  не единицу, а десятичную величину (0.5, 0.05, 0.01). Но для использования этого метода нужно знать, как именно мы выбираем к.

Бэк-офф и Интерполяция. Бэк-офф: если нет доказательств, что встречается триграмма, мы сужаем контекст до биграммы или даже униграммы (без доказательств не выбираем н-грамму больщого порядка). Таким образом мы "отступаем", пока не найдётся н-грамма с ненулевой вероятностью (т.е. которая уже встречалась в тренировочной выборке). С наиболее вероятных слов убирается немного вероятности, чобы у менее вероятных слов тоже был некоторый вес (т.к. они всё же могут встречаться)
Интерполяция: сочетаем оценки вероятности всех н-грамм, комбинируя триграмму, биграмму и униграмму (например, с помощью линейной интерполяции).

Сглаживание Кнезера-Нея - самое эффективное. Заключается в том, чтобы из каждого показателя вычесть фиксированное чилсло (можно ли таким образом перевести absolute discounting как "абсолютное вычитание" не уверена:)). Также при этом сглаживании учитывается контекст (если слово появлялось во множестве контекстов, тем вероятней, что оно появится в новом).

Глупый бэк-офф (глупое отступление?) - не стремится к распределению вероятности. Действует по факту: если н-грамма высокого порядка не встречается, мы отступаем к н-грамме предыдущего порядка, которой соответствует фиксированный вес вне контекста. 
